# Agent 2: Forecaster
**`Forecaster`** is an agent focused on forecasting future electricity consumption plan based on historical data.

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# import the necessary libraries
import os
import json
import pandas as pd
from typing import Any, Optional, Literal, Set, Callable
from nixtla import NixtlaClient

In [3]:
# Define the forecast_consumption function
def forecast_consumption() -> str:
    """
    Forecast electricity consumption        
    :return: json of forecasted consumption
    
    :rtype: str
    """
    
    #Initialize the Nixtla client with the TimeGEN API key and endpoint
    
    nixtla_client = NixtlaClient(
        base_url=os.getenv("TIME_GEN_ENDPOINT"),
        api_key=os.getenv("TIME_GEN_KEY"),
    )
    consumption = pd.read_csv('./data/consumption.csv')
    forecasted_consumption = nixtla_client.forecast(df=consumption, h=12, freq='MS', time_col='month', target_col='consumption')               
    forecasted_consumption['month'] = forecasted_consumption['month'].astype(str)
    
    return forecasted_consumption.to_json(orient='records')

In [4]:
import os
import logging
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool

# Create a project client

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


In [5]:
from typing import Any, Set, Callable
from azure.ai.projects.models import FunctionTool

# Define function tool with the forecast_consumption function
user_functions: Set[Callable[..., Any]] = {forecast_consumption}
functions = FunctionTool(functions=user_functions)
print(functions.definitions)

[{'type': 'function', 'function': {'name': 'forecast_consumption', 'description': 'Forecast electricity consumption        ', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}]


In [7]:
import logging
# Set the logging level for the Azure SDK
logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

In [8]:
# Create an agent 
data_analyzer = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name="forecast",
    description="An agent that forecasts electricity consumption",
    instructions="Hello, you are helpful assistant who answers question on future electricity consumption.",
    tools=functions.definitions,
    # Parameters
    temperature=0.5,
    top_p=0.95,
    
)

print(f"Created agent, agent ID: {data_analyzer.id}")

Created agent, agent ID: asst_01PlD0q8fH1Yzdg4qde8WVB4


In [9]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello, what would be my electricity consumption in next 12 months."
)

# Create and process assistant run in thread with tools
run = project_client.agents.create_run(
    thread_id=thread.id, assistant_id=data_analyzer.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(
                thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")

Created thread, ID: thread_tq54nD7mYKFMVDUjhhqxp2VR
Created run, ID: run_eRvDSydAdWnK7cKUORqdfNZz
Executing tool call: {'id': 'call_O0SaXiXkYeRkJ6BAr99l6L9U', 'type': 'function', 'function': {'name': 'forecast_consumption', 'arguments': '{}'}}


c:\Users\shilpajain\AppData\Local\Programs\Python\Python312\Lib\site-packages\nixtla\nixtla_client.py:867: UserWarning: Azure endpoint detected, setting `model` to 'azureai'.
  warnings.warn("Azure endpoint detected, setting `model` to 'azureai'.")
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Tool outputs: [{'tool_call_id': 'call_O0SaXiXkYeRkJ6BAr99l6L9U', 'output': '[{"month":"2025-01-01","TimeGPT":635.0912475586},{"month":"2025-02-01","TimeGPT":570.1800537109},{"month":"2025-03-01","TimeGPT":626.5328369141},{"month":"2025-04-01","TimeGPT":649.0748901367},{"month":"2025-05-01","TimeGPT":661.8880004883},{"month":"2025-06-01","TimeGPT":679.6558837891},{"month":"2025-07-01","TimeGPT":634.1744995117},{"month":"2025-08-01","TimeGPT":661.2553710938},{"month":"2025-09-01","TimeGPT":667.4523925781},{"month":"2025-10-01","TimeGPT":609.4385986328},{"month":"2025-11-01","TimeGPT":725.0209960938},{"month":"2025-12-01","TimeGPT":644.9719238281}]'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [14]:
# Helper function

from datetime import datetime
def get_conversation_md(conversation):
    """
    Function to return a conversation in Markdown (MD) format as a string.
    """
    messages = conversation.get("data", [])
    if not messages:
        return "No messages found in the conversation."

    # Initialize a list to hold the Markdown lines
    md_lines = []
    md_lines.append("# Conversation")
    md_lines.append("___")  # Markdown horizontal line

    # Iterate through the messages
    # Reversing to maintain chronological order
    for message in reversed(messages):
        role = message.get("role", "unknown").capitalize()
        timestamp = message.get("created_at")
        content = message.get("content", [])

        # Convert timestamp to a readable format
        if timestamp:
            timestamp = datetime.fromtimestamp(
                timestamp).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')
        else:
            timestamp = "Unknown time"

        # Extract the text content
        message_text = ""
        for item in content:
            if item.get("type") == "text":
                message_text += item["text"].get("value", "")

        # Append the message in Markdown format
        md_lines.append(f"### **{role}** ({timestamp})")
        md_lines.append(f"{message_text}")
        md_lines.append("___")  # Markdown horizontal line

    # Join the lines with newlines to form the complete Markdown string
    return "\n".join(md_lines)

In [16]:
from IPython.display import Markdown, display

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(get_conversation_md(messages)))

# Conversation
___
### **User** (2025-02-08 00:28:37 Malay Peninsula Standard Time)
Hello, what would be my electricity consumption in next 12 months.
___
### **Assistant** (2025-02-08 00:28:54 Malay Peninsula Standard Time)
Here is the forecast of your electricity consumption for the next 12 months:

- January 2025: 635.09 kWh
- February 2025: 570.18 kWh
- March 2025: 626.53 kWh
- April 2025: 649.07 kWh
- May 2025: 661.89 kWh
- June 2025: 679.66 kWh
- July 2025: 634.17 kWh
- August 2025: 661.26 kWh
- September 2025: 667.45 kWh
- October 2025: 609.44 kWh
- November 2025: 725.02 kWh
- December 2025: 644.97 kWh

This forecast is based on historical data and other relevant factors.
___